In [1]:
import tensorflow as tf

In [2]:
import cv2

In [3]:
import dlib

In [4]:
from keras.models import Model
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [5]:
import numpy as np

In [6]:
# Images Dimensions
img_width, img_height = 200, 200

In [7]:
train_data_dir = 'Downloads/video/data/training'
validation_data_dir = 'Downloads/video/data/validation'

In [8]:
nb_train_samples = 5552
nb_validation_samples = 1000
epochs = 50
batch_size = 128

In [9]:
callbacks = TensorBoard(log_dir='./grafik')

In [10]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [11]:
# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [12]:
# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 5552 images belonging to 5 classes.


In [13]:
# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1000 images belonging to 5 classes.


In [14]:
# Feature Extraction Layer
inputs = Input(shape=(img_width, img_height, 3))
conv_layer = Conv2D(16, (5, 5), strides=(3,3), activation='relu')(inputs) 
conv_layer = ZeroPadding2D(padding=(1,1))(conv_layer) 
conv_layer = Conv2D(32, (5, 5), strides=(3,3), activation='relu')(conv_layer) 
conv_layer = MaxPooling2D((2, 2))(conv_layer) 
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer) 
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer)

In [15]:
# Flatten Layer
flatten = Flatten()(conv_layer)

In [16]:
# Fully Connected Layer
fc_layer = Dense(128, activation='relu')(flatten)
fc_layer = Dense(64, activation='relu')(fc_layer)
fc_layer = Dense(32, activation='relu')(fc_layer)
outputs = Dense(5, activation='softmax')(fc_layer)

In [17]:
model = Model(inputs=inputs, outputs=outputs)

In [18]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [19]:
# Print Model Summary
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 66, 66, 16)        1216      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 68, 68, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          3692

In [20]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size, 
    callbacks=[callbacks])


Epoch 1/50
43/43 [==============================] - 121s 3s/step - loss: 1.6055 - acc: 0.2911 - val_loss: 1.5089 - val_acc: 0.3984
Epoch 2/50
43/43 [==============================] - 119s 3s/step - loss: 1.3067 - acc: 0.5100 - val_loss: 0.8528 - val_acc: 0.7087
Epoch 3/50
43/43 [==============================] - 118s 3s/step - loss: 0.5976 - acc: 0.7968 - val_loss: 0.0946 - val_acc: 0.9702
Epoch 4/50
43/43 [==============================] - 118s 3s/step - loss: 0.4006 - acc: 0.8934 - val_loss: 0.2105 - val_acc: 0.9897
Epoch 5/50
43/43 [==============================] - 117s 3s/step - loss: 0.1251 - acc: 0.9685 - val_loss: 0.1905 - val_acc: 0.9220
Epoch 6/50
43/43 [==============================] - 118s 3s/step - loss: 0.2584 - acc: 0.9346 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 7/50
43/43 [==============================] - 120s 3s/step - loss: 0.1065 - acc: 0.9742 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 8/50
43/43 [==============================] - 117s 3s/step - loss: 0.5662 - a

In [21]:
model.save_weights('wajah.h5')

In [22]:
%load_ext tensorboard

In [24]:
tensorboard --logdir grafik/

Reusing TensorBoard on port 6006 (pid 2764), started 0:01:23 ago. (Use '!kill 2764' to kill it.)

In [28]:
model.load_weights('wajah.h5')

In [46]:
test_data_dir = 'Downloads/video/data/test'

In [47]:
generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='binary',
        shuffle=False)

Found 1000 images belonging to 1 classes.


In [48]:
predict = model.predict(generator)

In [36]:
import pandas as pd

In [49]:
face = pd.DataFrame(predict)

In [50]:
print(predict)

[[1.42874077e-08 6.00275143e-07 1.04027143e-11 2.26475103e-10
  9.99999404e-01]
 [3.66769393e-10 3.24700835e-08 6.36293115e-14 3.77352527e-13
  1.00000000e+00]
 [1.51093160e-09 6.26804280e-08 1.74357517e-13 7.76214054e-13
  9.99999881e-01]
 ...
 [1.00000000e+00 9.96670225e-26 0.00000000e+00 4.42271756e-32
  8.37702518e-36]
 [1.00000000e+00 1.02244854e-25 0.00000000e+00 2.86523222e-32
  5.87987024e-36]
 [1.00000000e+00 8.35722320e-24 4.39414912e-38 2.76525352e-29
  1.20539580e-33]]


In [53]:
face.columns = [ 'Nessie Judge', 'David', 'Hansol', 'Arnold', 'Joko Widodo' ]
print(face)

     Nessie Judge         David        Hansol        Arnold   Joko Widodo
0    1.428741e-08  6.002751e-07  1.040271e-11  2.264751e-10  9.999994e-01
1    3.667694e-10  3.247008e-08  6.362931e-14  3.773525e-13  1.000000e+00
2    1.510932e-09  6.268043e-08  1.743575e-13  7.762141e-13  9.999999e-01
3    6.382915e-10  1.336836e-08  1.476896e-14  1.193507e-14  1.000000e+00
4    6.565774e-11  3.210286e-09  6.888510e-16  1.079547e-15  1.000000e+00
..            ...           ...           ...           ...           ...
995  1.000000e+00  8.677511e-25  0.000000e+00  1.247182e-30  1.152581e-34
996  1.000000e+00  1.758156e-24  0.000000e+00  3.081274e-30  2.408811e-34
997  1.000000e+00  9.966702e-26  0.000000e+00  4.422718e-32  8.377025e-36
998  1.000000e+00  1.022449e-25  0.000000e+00  2.865232e-32  5.879870e-36
999  1.000000e+00  8.357223e-24  4.394149e-38  2.765254e-29  1.205396e-33

[1000 rows x 5 columns]


In [54]:
face.to_excel("face.xlsx")  

In [55]:
import argparse as arg